# Part 0 - Libraries and Geoinfo file upload

## Available data analysis NYC information

In [ ]:
#first import needed libraries
import numpy as np # numpy arrays
import pandas as pd # dataframes and many many other things
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # work with json files
import requests #make requests to urls

import urllib #open urls

! pip install folium
import folium

# installed shapely library from a wheel file - https://www.lfd.uci.edu/~gohlke/pythonlibs/
# instruction to install library into conda using cmd - https://www.geeksforgeeks.org/python-add-packages-to-anaconda-environment/
!pip install shapely
import shapely
import shapely.wkt

from shapely import geometry

import geopy.distance

import ast # for transforming strings to the list/tuple

First we take data related to the New-york city (NYC from now on) Neighborhood tabulation areas (NTAs) <br>
For this I've searched the internet and got to the valuable site, and this directory https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page <br>
https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Neighborhood-Tabulatio/swpk-hqdp/data <br>
here we've found population amounts by NTAs, so we definetely can calculate the population density based on this information. <br>
and income information by NTA 'http://www.city-data.com/nbmaps/neigh-New-York-New-York.html'
'https://statisticalatlas.com/place/New-York/New-York/Household-Income#figure/neighborhood'

In [ ]:
url_geoinfo='http://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/nynta/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=geojson'
with urllib.request.urlopen(url_geoinfo) as ref_geoinfo:
    json_geoinfo=json.load(ref_geoinfo)

json_geoinfo.keys()

# Part 1 - preparation of open source New-york city data

So, we've stumbled upon the infromation regarding the shapes of the NYC NTAs <br>
Luckily, I've found __shapely__ library to work with the shape objects, understand if some venue (ll point) is inside a shape, <br> 
calculate area sizes and find centroids of the NTAs

In [ ]:
# after research of the json file's structure, let's see one element example
print(json_geoinfo['features'][0].keys())
print(json_geoinfo['features'][0]['properties'].keys())

In [ ]:
# now let's create a dataframe that holds all the shapes and desired infromation regarding the shapes
shape_tlist=[]
shape_clist=[]
shape_ntalist=[]
shape_arealist=[]
shape_countlist=[]
for i,elem in enumerate(json_geoinfo['features']):
    shape_type=elem['geometry']['type']
    shape_tlist.append(shape_type)
    shape_ntalist.append(elem['properties']['NTACode'])
    shape_arealist.append(elem['properties']['Shape__Area']/10000000)
    #print(i,shape_type)
    shape_coord=elem['geometry']['coordinates'];
    if shape_type=='Polygon':
        shape_obj=geometry.Polygon(shape_coord[0])
        i=1
    elif shape_type=='MultiPolygon':
        #print(shape_coord[1])
        shape_multi=[]
        i=0
        for shape_elem in shape_coord:
            for shape_detail in shape_elem:
                i+=1
                shape_multi.append(geometry.Polygon(shape_detail))
        shape_obj=geometry.MultiPolygon(shape_multi)
    else:
        shape_obj='NA'
    shape_clist.append(shape_obj)
    shape_countlist.append(i)
df_NTA=pd.DataFrame(zip(shape_ntalist,shape_tlist, shape_countlist,shape_clist,shape_arealist), columns=['NTA Code','Shape type','Poly Count','geometry','Shape area size, km'])
df_NTA.set_index('NTA Code',inplace=True)
df_NTA.tail()

In [ ]:
df_NTA['geometry'].loc['BX99']
#that's how a shapely multipolygon object is displayed - in comparison with the map, this is the parks in bronx

## This concludes our first task - to extract the infromation regarding how the neighborhoods look like
Next, we're going to calculate based on the shapes, what is the square area size, what are the centroids of the noted shapes, and what are the max distance from every shape point to it's centroid (we're gonna need it when we'll extract the data from FourSquare)

In [ ]:
# also there is an error between built-in shape area size and polygon area simply because the Earth isn't flat, so you need to keep in mind of such inaccuracy and how it can affect your calculation
df_NTA['Polygon area, km']=df_NTA['geometry'].apply(lambda x:x.area*10000)

#second - let's point out centroids (or a list of centroids for polygons)
def multi_centroid(multi_shape):
    cent_list=[]
    for poly in multi_shape:
        cent_list.append(poly.centroid.coords[0])
    return cent_list

df_NTA.loc[df_NTA['Poly Count']>1,'Shape centroid']=df_NTA.loc[df_NTA['Poly Count']>1,'geometry'].apply(lambda x:multi_centroid(x))
df_NTA.loc[df_NTA['Poly Count']==1,'Shape centroid']=df_NTA.loc[df_NTA['Poly Count']==1,'geometry'].apply(lambda x:x.centroid.coords[0])

#third - let's calculate the the farthest point inside each polygon from the centroid, and that we did with the notion of the earth parameters, even if we've made the algorithm a little longer by it

def farthest_determinator(poly_shape):
    cent=poly_shape.centroid
    max=0
    for ll in poly_shape.exterior.coords:
        if max<geopy.distance.geodesic(ll, (cent.x,cent.y)).km:
            max=geopy.distance.geodesic(ll, (cent.x,cent.y)).km
    return max

def multi_far(multi_shape):
    max_list=[]
    for poly in multi_shape:
        max_list.append(farthest_determinator(poly))
    return max_list

df_NTA.loc[df_NTA['Poly Count']>1,'Max radius, km']=df_NTA.loc[df_NTA['Poly Count']>1,'geometry'].apply(lambda x:multi_far(x))
df_NTA.loc[df_NTA['Poly Count']==1,'Max radius, km']=df_NTA.loc[df_NTA['Poly Count']==1,'geometry'].apply(lambda x:farthest_determinator(x))

df_NTA.head()

# Now we're taking the information regarding the population size by NTA's from another source
Unfortunately, the latest infrormaton available is 2010, but We'll try to look for additional infromation later on

In [ ]:
url_population='https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv'
df_population=pd.read_csv(url_population)
df_population=df_population.loc[df_population['Year']==2010]
df_population.reset_index(drop=True,inplace=True)
df_population.set_index('NTA Code',inplace=True)

df_merged=df_population.join(df_NTA,on='NTA Code')
df_merged.head()

In [ ]:
df_borough_pop=df_merged[['Shape area size, km','Polygon area, km','Borough']].groupby(by='Borough').sum()
# Polygon area had been calculated in order to check the reasonableness of the default built-in shape area size information, however the differences are caused by
# the fact that you cannot just calculate the area size of 2D object, because the Earth is not flat
df_borough_pop

In [ ]:
#df_borough_pop.drop('Polygon area, km',axis=1,inplace=True)
df_merged.drop('Polygon area, km',axis=1,inplace=True)

## In the next step, we're gonna look for additional information sourses and check the provided data quality based on external information
Like - the borough area sizes from wikipedia

In [ ]:
# now we double check the data with aggregated Borough information available (couldn't find information for every NTA)
from bs4 import BeautifulSoup
url_wiki='https://en.wikipedia.org/wiki/Boroughs_of_New_York_City'
wiki_html=urllib.request.urlopen(url_wiki).read()
wiki_soup=BeautifulSoup(wiki_html,'html.parser')
#wiki_soup.prettify()

In [ ]:
soup_table = wiki_soup.find('table',{'class':'wikitable sortable'})
#so then, after we locate the table, we only take that relevant part, and find every text line by searching 'td'
soup_links=soup_table.findAll('td')
columns=['Borough', 'County', 'Population','GDP US bln','GDP per capita','Area sq ml','Area sq km','Density sq ml','Density sq km']
NYC_borough_stat=pd.DataFrame(columns=columns)
temp_dict={}
for n,line in enumerate(soup_links):
#basically, every element in 'soup_links' is every cell in the table, and as we know there are 9 elements in every row, so we are configuring this algorithm to
#re-write a dictionary 'temp_dict' on every 9-rd iteration and append it to the created dataframe 'postal_codes'
    curr=line.get_text().strip('\n') # at this line - we're getting only relevant text without any html symbols and other things, also we're stripping the next line symbol
#this if is served for multi-purpose 1. if n%9==0 then the we've got the full temp_dict recorded that we need to record
# however if n==0, then we're at the start of the loop, and we don't have anything to write yet
# also it is given at the task that we need to delete all unassigned lists
    if (n%9==0) and (n!=0) and (temp_dict['Borough']!='Not assigned'):
# next if is to see when the Borough already had been recorded, so if the loc method with condition inside shows that it's empty - then we need to write a new line
        if NYC_borough_stat.loc[NYC_borough_stat['Borough']==temp_dict['Borough']].empty:
            NYC_borough_stat=NYC_borough_stat.append(temp_dict,ignore_index=True)
    temp_dict[columns[n%9]]=curr
    
# if a neighbourhood not assigned, then it must be replaced
NYC_borough_stat['Borough'].loc[0]='Bronx'
NYC_borough_stat.set_index('Borough',inplace=True)
NYC_borough_stat.head()

In [ ]:
# it seems like have some differences, and in order to minimize them, 
# let's create a normalizing square area coefficients for each borough and then apply it to the NTA information
df_borough_pop['stat area sq km']=NYC_borough_stat['Area sq km']
df_borough_pop['area_coef']=df_borough_pop.apply(lambda x: float(x['stat area sq km'])/float(x['Shape area size, km']), axis=1)
df_borough_pop

In [ ]:
try:
    df_merged.drop('area_coef',axis=1,inplace=True)
except:
    print('no area_coef column present_yet')
df_merged=df_merged.join(df_borough_pop['area_coef'],on='Borough')
df_merged['Area_normalized, sq km']=df_merged.apply(lambda x: float(x['area_coef'])*x['Shape area size, km'], axis=1)
df_merged.drop(['area_coef','Shape area size, km'],axis=1,inplace=True)
df_merged['Population_density']=df_merged.apply(lambda x: int(x['Population'])/float(x['Area_normalized, sq km']),axis=1)
df_merged.head()

## Now, let's see aditional sourse of information regarding the income, population and other

In [ ]:
import zipfile
import io

zip_url='https://s3.amazonaws.com/geoda/data/nycnhood_acs.zip'
zip_bytes=urllib.request.urlopen(zip_url).read()
#this is the zip file we need to open and iterate through it's contents
with zipfile.ZipFile(io.BytesIO(zip_bytes)) as my_zip_file:
    zinfo=my_zip_file.infolist()
    znames=my_zip_file.namelist()
znames
# so, now we can see, that we need to open the dbf file inside the zip file and shape object to see if they have more accurate data
# 2 files - 'NYC_Nhood ACS2008_12.dbf', 'NYC_Nhood ACS2008_12.shp', number 0 and 5

In [ ]:
!pip install simpledbf
!pip install pyshp
from simpledbf import Dbf5
import shapefile

with zipfile.ZipFile(io.BytesIO(zip_bytes)) as my_zip_file:
    dbf_ref=my_zip_file.extract(zinfo[0])
    dbf=Dbf5(dbf_ref)
    df_dbf = dbf.to_dataframe()
    shp_ref=my_zip_file.extract(zinfo[5])
    shp = shapefile.Reader(shp_ref)
print(shp.fields)
df_dbf.head()
#zinfo[5]

In [ ]:
alt_shapelist=[]
alt_shape_ntas=[]
alt_shape_boronames=[]
alt_shape_type=[]
alt_shape_count=[]
for feature in shp.shapeRecords():
    alt_shape_ntas.append(feature.record[-7]) # this is the information file with details - however all of them we took from dbf file, here we're linking only the NTA code"
    alt_shape_boronames.append(feature.record[-9])
    shape_obj=shapely.geometry.shape(feature.shape)
    alt_shapelist.append(shape_obj)
    i=1
    if isinstance(shape_obj,shapely.geometry.polygon.Polygon):
        i=1
        shp_type='Polygon'
    else:
        for x in shape_obj:
            i+=1
        shp_type='MultiPolygon'
    alt_shape_type.append(shp_type)
    alt_shape_count.append(i)
df_alt_NTA=pd.DataFrame(zip(alt_shape_ntas,alt_shape_boronames, alt_shapelist,alt_shape_type,alt_shape_count), columns=['NTA Code','Borough','geometry','Shape type','Poly Count'])
df_alt_NTA.set_index('NTA Code',inplace=True)
df_alt_NTA['alt Polygon area, km']=df_alt_NTA['geometry'].apply(lambda x:x.area*10000)
df_alt_NTA.head()

In [ ]:
df_borough_alt=df_alt_NTA[['alt Polygon area, km','Borough']].groupby(by='Borough').sum()
df_borough_pop2=df_borough_pop.join(df_borough_alt)
df_borough_pop2
#df_dbf=df_dbf.join(df_alt_NTA,on='ntacode')
#df_dbf.head()

So now we know that these polygons are somehow different, however from the polygon area size accuracy point of view, they are very much alike, so let's keep the old one for now

In [ ]:
url_alt_info='https://geodacenter.github.io/data-and-lab//NYC-Nhood-ACS-2008-12/'
alt_html=urllib.request.urlopen(url_alt_info).read()
alt_soup=BeautifulSoup(alt_html,'html.parser')
#print(alt_soup.prettify())

soup_table = alt_soup.find('table')
#so then, after we locate the table, we only take that relevant part, and find every text line by searching 'td'
soup_links=soup_table.findAll('td')
columns=['Variable','Meaning']
alt_variables=pd.DataFrame(columns=columns)
temp_dict={}
for n,line in enumerate(soup_links):
#basically, every element in 'soup_links' is every cell in the table, and as we know there are 9 elements in every row, so we are configuring this algorithm to
#re-write a dictionary 'temp_dict' on every 9-rd iteration and append it to the created dataframe 'postal_codes'
    curr=line.get_text().strip('\n') # at this line - we're getting only relevant text without any html symbols and other things, also we're stripping the next line symbol
#this if is served for multi-purpose 1. if n%9==0 then the we've got the full temp_dict recorded that we need to record
# however if n==0, then we're at the start of the loop, and we don't have anything to write yet
# also it is given at the task that we need to delete all unassigned lists
    if (n%2==0) and (n!=0):
# next if is to see when the Borough already had been recorded, so if the loc method with condition inside shows that it's empty - then we need to write a new line
        if alt_variables.loc[alt_variables['Variable']==temp_dict['Variable']].empty:
            alt_variables=alt_variables.append(temp_dict,ignore_index=True)
    temp_dict[columns[n%2]]=curr
alt_variables=alt_variables.loc[alt_variables['Variable'].isin(['medianinco',
                                                                 'popinlabou',
                                                                 'ntacode',
                                                                 'boroname',
                                                                 'popdty',
                                                                 'medianage',
                                                                 'poptot'])]
alt_variables=alt_variables.append(pd.DataFrame([['UEMPRATE','Some coeffient, that is used for some relative data like median age and income']],columns=['Variable','Meaning']))

In [ ]:
df_dbf2=df_dbf.loc[:,alt_variables['Variable'].tolist()]
df_dbf2.columns=['poptot','popinlabou', 'Borough', 'popdty','NTA Code','medianinco', 'medianage','UEMPRATE']
df_dbf2.sort_values(by='NTA Code',inplace=True)
def mile_to_km(x):
    if x=='NA':
        return 'NA'
    else:
        return float(x)/2.58999
def floater(x):
    if x=='NA':
        return 'NA'
    else:
        return float(x)
    
df_dbf2['popdty, per sq km']=df_dbf2.loc[:,'popdty'].apply(mile_to_km)
df_dbf2['medianage']=df_dbf2.loc[:,'medianage'].apply(floater)
df_dbf2['medianinco']=df_dbf2.loc[:,'medianinco'].apply(floater)
df_dbf2.drop('popdty',axis=1,inplace=True)
df_dbf2.set_index('NTA Code',inplace=True)
print(df_dbf2.sum())
df_dbf2.head()

## Now we see an error with the data presented - it obviously has faulty information - population density needs to be divided by 10 at least, median age is somewhere skyhigh and the median income is also showing inconsistency
For the purpose of this test, let's assume that this data (and with gini and other parameters are somehow hinting this) had been just normalized for the purpose of some model creation. <br>
And our task is to make it readable and to denormalize it - and get the estimated readable amounts that looks like median income and median age.<br>
So, first, let's calculate the weighted average amounts and the min and max values to see the situation <br>
Second, we're going to compare it with the median values for age and income for NYC and it's boroughs, and set up the linear transformation, that will change the current values

In [ ]:
def weighted_income(x):
    if x['medianinco']=='NA' or x['poptot']=='NA' or x['poptot']==0:
        return np.NaN
    else:
        return float(x['medianinco'])*float(x['poptot'])/8199221

def weighted_age(x):
    if x['medianage']=='NA' or x['poptot']=='NA' or x['poptot']==0:
        return np.NaN
    else:
        return float(x['medianage'])*float(x['poptot'])/8199221

def median_income(x):
    if x['medianinco']=='NA' or x['poptot']=='NA' or x['poptot']==0:
        return np.NaN
    else:
        return float(x['medianinco'])

def median_age(x):
    if x['medianage']=='NA' or x['poptot']=='NA' or x['poptot']==0:
        return np.NaN
    else:
        return float(x['medianage'])

def labour_calc(x):
    if x['popinlabou']=='NA' or x['poptot']=='NA' or x['poptot']==0:
        return np.NaN
    else:
        return float(x['popinlabou'])/float(x['poptot'])

df_dbf2['weighted_income_median_avg']=df_dbf2.apply(weighted_income, axis=1)
df_dbf2['weighted_age_median_avg']=df_dbf2.apply(weighted_age, axis=1)
df_dbf2['medianinco']=df_dbf2.apply(median_income, axis=1)
df_dbf2['medianage']=df_dbf2.apply(median_age, axis=1)
df_dbf2['labour_coef']=df_dbf2.apply(labour_calc, axis=1)
df_dbf2.head()

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

pd.options.display.float_format = '{:.2f}'.format
print(color.BOLD + 'Minimum amounts' + color.END)
print(df_dbf2[['medianinco','medianage']].min())
print(color.BOLD + '\nAggregated weighted average info' + color.END)
print(df_dbf2[['poptot','weighted_income_median_avg','weighted_age_median_avg']].sum())
print(color.BOLD + '\nMaximum amounts' + color.END)
print(df_dbf2[['medianinco','medianage']].max())

### Based on the information above and wikipedia averages for NYC overall and boroughs, I've manually selected linear transformation parameters

In [ ]:
df_dbf2['medianinco']=df_dbf2.loc[:,'medianinco'].apply(lambda x: 30000+x/36)
df_dbf2['medianage']=df_dbf2.loc[:,'medianage'].apply(lambda x: 30+x/100)

df_dbf2['weighted_income_median_avg']=df_dbf2.apply(weighted_income, axis=1)
df_dbf2['weighted_age_median_avg']=df_dbf2.apply(weighted_age, axis=1)

print(color.BOLD + 'Minimum amounts' + color.END)
print(df_dbf2[['medianinco','medianage']].min())
print(color.BOLD + '\nAggregated weighted average info' + color.END)
print(df_dbf2[['poptot','weighted_income_median_avg','weighted_age_median_avg']].sum())
print(color.BOLD + '\nMaximum amounts' + color.END)
print(df_dbf2[['medianinco','medianage']].max())

In [ ]:
df_dbf2=df_dbf2.loc[:,['medianinco','medianage','popinlabou','labour_coef']]
df_dbf2.head()

In [ ]:
try:#this thing is needed only to run this cell without needing previous iterations
    df_merged.drop(['medianinco','medianage','labour_coef'],axis=1,inplace=True)
except:
    print('No need to delete columns')

try:#this thing the same as above
    df_merged.set_index('NTA Code',inplace=True)
except:
    print('No need to set index')

df_merged=df_merged.join(df_dbf2)
df_merged.reset_index(inplace=True)
df_merged.head()

In [ ]:
# please note, that there is a confirmed error with Google Chrome browser showing folium maps - that there is a problem displaying 80+ elements on the map - it just shows emplty maps
# also there is an example - 'https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb?flush_cache=true'
# where you can use GeoJson instead of choropleth, however for this task choropleth and mozilla combination works fine

# define the world map
world_map = folium.Map(location=[40.7128,-74.0060], zoom_start=12)
#please be aware that location=[latitude, longitude] is encoded in folium, however GEOjson files have [longitude,latitude] written down
folium.Choropleth(
     name='Population density layer',
     geo_data=json_geoinfo,
     data=df_merged,
     columns=['NTA Code', 'Population_density'],
     key_on='feature.properties.NTACode',
     fill_color='YlOrRd', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Population density by NTA'
).add_to(world_map)

folium.Choropleth(
     name='Median income layer',
     geo_data=json_geoinfo,
     data=df_merged,
     columns=['NTA Code', 'medianinco'],
     key_on='feature.properties.NTACode',
     fill_color='BuGn', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Median income amount by NTA'
).add_to(world_map)

folium.Choropleth(
     name='Median age layer',
     geo_data=json_geoinfo,
     data=df_merged,
     columns=['NTA Code', 'medianage'],
     key_on='feature.properties.NTACode',
     fill_color='RdPu', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Median age by NTA'
).add_to(world_map)

folium.Choropleth(
     name='Working people percentage',
     geo_data=json_geoinfo,
     data=df_merged,
     columns=['NTA Code', 'labour_coef'],
     key_on='feature.properties.NTACode',
     fill_color='RdBu', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Share of working population'
).add_to(world_map)


folium.LayerControl().add_to(world_map)

world_map

### Please keep in mind that the following map has layers - switch through them in order to see various stats on the map
We also see that the median age and median income look very much alike - sp I think it would be wise to drop the age parameter - because similar, or correlated parameters are usually bad for model creation

In [ ]:
#I'm saving this file as csv on local computer in order to have the step 1 end-result ready without running the whole programm
df_merged.to_csv(r'NYC_current_data.csv')

# Part 2 - extraction of Foursquare data

## So now we have all external data sorted out, it is the time for us to get the foursquare data to be extracted - we're going to go through all the neighborhoods and extract all the venues in the max radius amount
Limit is going to be calculated based on the radius amount

In [ ]:
CLIENT_ID = 'FICA04YHTU00Y1DHOH1T5VALLE2VGNRGTJJBVB0ECZ4QEE1C' # your Foursquare ID
CLIENT_SECRET = 'LFO0GLMBQQQTCB1TCBSDFMKLRDGGEQK4O1VW5YLN4SR1NPHK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#first we read some data
df_merged=pd.read_csv('NYC_current_data.csv')
df_merged.drop('Unnamed: 0',axis=1,inplace=True)
df_merged['geometry']=df_merged.loc[:,'geometry'].apply(lambda x:  shapely.wkt.loads(x))
df_merged['Shape centroid']=df_merged.loc[:,'Shape centroid'].apply(lambda x: ast.literal_eval(x))
df_merged['Max radius, km']=df_merged.loc[:,'Max radius, km'].apply(lambda x: ast.literal_eval(x))
df_merged.head(7)

In [ ]:
# this function doesn't take
def getNearbyVenues(row_series):
    url='https://api.foursquare.com/v2/venues/search'
    NTA_Code=row_series[0]
    shapely_point=row_series[1]
    max_radius=float(row_series[2])*1000
    NTA_geometry=row_series[3]
    lng=round(shapely_point[0],6)
    lat=round(shapely_point[1],6)
    params={
            'client_id':CLIENT_ID,
            'client_secret':CLIENT_SECRET,
            'v':VERSION,
            'intent':'browse',
            'll':str(lat)+','+str(lng),
            'limit':300, # in accordance with their API, limit is 50, however, let's try our luck
            'radius':int(max_radius) #convert km into meters
    }
    # make the GET request
    results = requests.get(url,params=params).json()['response']['venues']
    # return only relevant information for each nearby venue
    
    venues_list=[]
    for v in results:
        if not v['categories']:
            category_id=np.NaN
            category_name=np.NaN
        else:
            category_id=v['categories'][0]['id']
            category_name=v['categories'][0]['name']
        cur_tuple=(
            NTA_Code,
            NTA_geometry,
            shapely_point,
            v['name'], 
            v['location']['lat'],
            v['location']['lng'],
            category_id,
            category_name
        )
        venues_list.append(cur_tuple)

    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['NTA Code', 
                  'geometry',
                  'Shape centroid',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category ID',
                  'Venue Category Name']
    
    return(nearby_venues)

In [ ]:
# this cell had been created solely for testing purposes - in order not to spend all my API limit on debugging
pd.options.display.float_format = '{:.6f}'.format

test_NTA_Code=df_merged.loc[0,['NTA Code','Shape centroid','Max radius, km','geometry']]
#getNearbyVenues(test_NTA_Code)
testmulti_NTA_Code=pd.DataFrame(zip(df_merged.loc[6,'Shape centroid'],df_merged.loc[6,'Max radius, km'],df_merged.loc[6,'geometry']),columns=['Shape centroid','Max radius, km','geometry'])
testmulti_NTA_Code['NTA Code']=df_merged.loc[6,'NTA Code']
testmulti_NTA_Code=testmulti_NTA_Code[['NTA Code','Shape centroid','Max radius, km','geometry']]
for index_test, row_test in testmulti_NTA_Code.iterrows():
    if index_test==0:
        print(row_test)
        df_curr=getNearbyVenues(row_test)
    else:
        df_curr=df_curr.append(getNearbyVenues(row_test),ignore_index=True)
df_curr.tail()

In [ ]:
for index_elem, row_elem in df_merged.iterrows():
    if row_elem['Poly Count']==1:
        df_curr=getNearbyVenues(row_elem[['NTA Code','Shape centroid','Max radius, km','geometry']])
    else:
        multi_row_split=pd.DataFrame(zip(row_elem['Shape centroid'],
                                         row_elem['Max radius, km'],
                                         row_elem['geometry']),
                                     columns=['Shape centroid','Max radius, km','geometry'])
        multi_row_split['NTA Code']=row_elem['NTA Code']
        multi_row_split=multi_row_split[['NTA Code','Shape centroid','Max radius, km','geometry']]
        for index_single,row_single in multi_row_split.iterrows():
            if index_single==0:
                df_curr=getNearbyVenues(row_single)
            else:
                df_curr=df_curr.append(getNearbyVenues(row_single),ignore_index=True)
    if index_elem==0:
        df_venues=df_curr
    else:
        df_venues=df_venues.append(df_curr,ignore_index=True)
df_venues.tail()

In [ ]:
# now we'll save the extracted data and finish this part - because we have API requests limit on foursquare
df_venues.to_csv(r'NYC_venues_list.csv')

# Part 3 - transformaton of extracted from Foursquare data

## The next step is to confirm that these venues really inside the shapely objects, because the request had been formed to find all the venues in a circle with the center in the Polygon centroids (multiple circles for multipolygons) and radius determined as the farthest distance to the polygon element

In [ ]:
df_venues=pd.read_csv('NYC_venues_list.csv')
df_venues.drop('Unnamed: 0',axis=1,inplace=True)
df_venues['geometry']=df_venues.loc[:,'geometry'].apply(lambda x:  shapely.wkt.loads(x))
df_venues['Shape centroid']=df_venues.loc[:,'Shape centroid'].apply(lambda x: ast.literal_eval(x))
# let's delete not venues without categories
df_venues.dropna(inplace=True)
df_venues.head(7)

In [ ]:
df_venues.loc[df_venues['NTA Code']=='BX09'].head() # so this means that the multipolygon neighborhoods got inserted a distinct polygon inside that shape attached

In [ ]:
# with this cell, we're removing all the objects, that actually don't belong to the neighborhood - due to the foursquare cirle extraction
def condition_contain (row):
    try: 
        return row['geometry'].contains(shapely.geometry.Point(row['Venue Longitude'],row['Venue Latitude']))
    except:
        print(row['geometry'])
        print(row['Venue Longitude'])
        print(row['Venue Latitude'])
        return False

df_venues['condition']=df_venues.apply(condition_contain,axis=1)
df_venues=df_venues.loc[df_venues['condition']==True].drop('condition',axis=1).reset_index()
df_venues.tail()
#as you can see, we've deleted about 2000 Venues

### Now let's aggregate similar venue categories

In [ ]:
CLIENT_ID = 'FICA04YHTU00Y1DHOH1T5VALLE2VGNRGTJJBVB0ECZ4QEE1C' # your Foursquare ID
CLIENT_SECRET = 'LFO0GLMBQQQTCB1TCBSDFMKLRDGGEQK4O1VW5YLN4SR1NPHK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def recursion_cat_search(json_file,prev_list=[]):
    curr_list=prev_list+[(json_file['id'],json_file['name'])]
    df_curr=pd.DataFrame()
    if not json_file['categories']:
        curr_dict={}
        for i in range(len(curr_list)):
            curr_dict['category {}'.format(i)]=curr_list[i]
        df_curr=df_curr.append(curr_dict,ignore_index=True,sort=False)
        return df_curr
    else:
        for item in json_file['categories']:
            df_curr=df_curr.append(recursion_cat_search(item,prev_list=curr_list),sort=False)
    return df_curr
cat_url='https://api.foursquare.com/v2/venues/categories'
params={
        'client_id':CLIENT_ID,
        'client_secret':CLIENT_SECRET,
        'v':VERSION,
}
cat_response=requests.get(cat_url,params=params).json()['response']
df_venue_cats=pd.DataFrame()
for cats in cat_response['categories']:
    df_venue_cats=df_venue_cats.append(recursion_cat_search(cats),sort=False)
df_venue_cats=df_venue_cats.reset_index()
df_venue_cats.head()

In [ ]:
import math
def id_extractor(elem):
    if type(elem) is not tuple:
        return np.NaN
    else:
        return elem[0]
# we need to modify this categories, to see only general and end-points - below code is old and not applicable
# def end_cat_finder(row):
#     if type(row['category 4']) is not tuple:
#         if type(row['category 3']) is not tuple:
#             if type(row['category 2']) is not tuple:
#                 if type(row['category 1']) is not tuple:
#                     return row['category 0']
#                 else:
#                     return row['category 1']
#             else:
#                 return row['category 2']
#         else:
#             return row['category 3']
#     else:
#         return row['category 4']

# df_venue_cats['category_final']=df_venue_cats.apply(end_cat_finder,axis=1)
# df_venue_cats['category ID']=df_venue_cats['category_final'].apply(lambda x: x[0])

df_venue_cats['General Category ID']=df_venue_cats['category 0'].apply(lambda x: x[0])
df_venue_cats['General Category name']=df_venue_cats['category 0'].apply(lambda x: x[1])
df_venue_cats['cat_0_ID']=df_venue_cats['category 0'].apply(id_extractor)
df_venue_cats['cat_1_ID']=df_venue_cats['category 1'].apply(id_extractor)
df_venue_cats['cat_2_ID']=df_venue_cats['category 2'].apply(id_extractor)
df_venue_cats['cat_3_ID']=df_venue_cats['category 3'].apply(id_extractor)
df_venue_cats['cat_4_ID']=df_venue_cats['category 4'].apply(id_extractor)
df_venue_cats.drop(['index','category 0','category 1','category 2','category 3','category 4'],axis=1,inplace=True)
df_venue_cats.tail()

In [ ]:
df_venue_cats[['General Category ID','General Category name','cat_4_ID']].dropna().set_index('cat_4_ID')

for i in range(5):
    cat_word='cat_{}_ID'.format(i)
    curr_cat=df_venue_cats[['General Category ID','General Category name',cat_word]].dropna().drop_duplicates().set_index(cat_word)
    print(curr_cat.shape)
    if i==0:
        df_venues_new=df_venues.join(curr_cat,on='Venue Category ID',how='inner')
    else:
        df_venues_new=df_venues_new.append(df_venues.join(curr_cat,on='Venue Category ID',how='inner'),ignore_index=True,sort=False)
    print(df_venues_new.shape)
df_venues_new.drop('index',inplace=True,axis=1)
df_venues_new.tail()

In [ ]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
NYC_food_venues=df_venues_new[df_venues_new['General Category name']=='Food']
NYC_food_map = folium.Map(location = [40.7128,-74.0060], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
Venue_loc = plugins.MarkerCluster().add_to(NYC_food_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(NYC_food_venues['Venue Latitude'], NYC_food_venues['Venue Longitude'], NYC_food_venues['Venue Category Name']):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(Venue_loc)

# display map
NYC_food_map

### Now let's aggregate the different categories (generalized)

In [ ]:
#df_venues_new[['NTA Code','General Category name','Venue']].groupby(by=['NTA Code','General Category name']).count()

NYC_onehot = pd.get_dummies(df_venues_new[['General Category name']], prefix="", prefix_sep="")
NYC_onehot['NTA Code']=df_venues_new['NTA Code']
Cols=NYC_onehot.columns.to_list()
Pos=Cols.index('NTA Code')
Cols=[Cols[Pos]]+Cols[:Pos]+Cols[Pos+1:]
NYC_onehot=NYC_onehot[Cols]
NYC_onehot.head()

In [ ]:
NYC_venue_grouped = NYC_onehot.groupby('NTA Code').sum().reset_index()
print(NYC_venue_grouped.shape)
NYC_venue_grouped.head()

In [ ]:
world_map = folium.Map(location=[40.7128,-74.0060], zoom_start=12)
#please be aware that location=[latitude, longitude] is encoded in folium, however GEOjson files have [longitude,latitude] written down
folium.Choropleth(
     name='College & University',
     geo_data=json_geoinfo,
     data=NYC_venue_grouped,
     columns=['NTA Code', 'College & University'],
     key_on='feature.properties.NTACode',
     fill_color='YlOrRd', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Number of Coll/Universities'
).add_to(world_map)

folium.Choropleth(
     name='Food',
     geo_data=json_geoinfo,
     data=NYC_venue_grouped,
     columns=['NTA Code', 'Food'],
     key_on='feature.properties.NTACode',
     fill_color='BuGn', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Number of Food joints'
).add_to(world_map)

# folium.Choropleth(
#      name='Nightlife Spot',
#      geo_data=json_geoinfo,
#      data=NYC_venue_grouped,
#      columns=['NTA Code', 'Nightlife Spot'],
#      key_on='feature.properties.NTACode',
#      fill_color='RdPu', 
#      fill_opacity=0.7, 
#      line_opacity=0.2,
#      legend_name='Number of Nightlife spots'
# ).add_to(world_map)

# folium.Choropleth(
#      name='Shop & Service',
#      geo_data=json_geoinfo,
#      data=NYC_venue_grouped,
#      columns=['NTA Code', 'Shop & Service'],
#      key_on='feature.properties.NTACode',
#      fill_color='YlOrRd', 
#      fill_opacity=0.7, 
#      line_opacity=0.2,
#      legend_name='Number of shops and services'
# ).add_to(world_map)

# folium.Choropleth(
#      name='Professional & Other Places',
#      geo_data=json_geoinfo,
#      data=NYC_venue_grouped,
#      columns=['NTA Code', 'Professional & Other Places'],
#      key_on='feature.properties.NTACode',
#      fill_color='BuGn', 
#      fill_opacity=0.7, 
#      line_opacity=0.2,
#      legend_name='Number of professional & other places'
# ).add_to(world_map)

# folium.Choropleth(
#      name='Travel & Transport',
#      geo_data=json_geoinfo,
#      data=NYC_venue_grouped,
#      columns=['NTA Code', 'Travel & Transport'],
#      key_on='feature.properties.NTACode',
#      fill_color='RdBu', 
#      fill_opacity=0.7, 
#      line_opacity=0.2,
#      legend_name='Number of travel & transport places'
# ).add_to(world_map)

folium.LayerControl().add_to(world_map)

world_map

### Subpart - merging venue and other data

In [ ]:
df_merged=pd.read_csv('NYC_current_data.csv')
df_merged.drop('Unnamed: 0',axis=1,inplace=True)
df_final=df_merged.join(NYC_venue_grouped.set_index('NTA Code'),on='NTA Code')
df_final.tail()

In [ ]:
df_final.to_csv(r'NYC_stat_aggregated.csv')

# Part 5 Data modeling

In [ ]:
#first we read some data
df_final=pd.read_csv('NYC_stat_aggregated.csv').set_index('NTA Code')
df_final.drop(['Unnamed: 0','Year','Max radius, km','FIPS County Code','Shape type','Poly Count','geometry','Shape centroid','Event'],axis=1,inplace=True)
df_final_clean=df_final.dropna()
print(df_final_clean.shape,df_final.shape)
df_final_clean.head()
# so if we delete all the rows without some of the data, we'll lose 23 NTA's

In [ ]:
!pip install sklearn

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split

y_data=df_final_clean[['Food']]
x_data=df_final_clean.drop(['Borough','NTA Name','Food'],axis=1)
#x_data=df_final_clean.drop(['NTA Code','Borough','NTA Name','Food'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)

In [ ]:
lr=linear_model.LinearRegression()

x_train_select=x_train[['Area_normalized, sq km']]
x_test_select=x_test[['Area_normalized, sq km']]
lr.fit(x_train_select,y_train)
np.set_printoptions(precision=4,formatter={"float_kind": lambda x: "%g" % x})
print(np.around(lr.coef_,decimals=4))

plt.figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(x_train_select, lr.coef_[0][0]*x_train_select + lr.intercept_[0], '-r')
plt.scatter(df_final_clean['Area_normalized, sq km'], df_final_clean['Food'], color='blue')
plt.xlabel("Area_normalized, sq km")
plt.ylabel("Food Venues count")
print('R^2 for the test data is {}'.format(lr.score(x_test_select, y_test)))
plt.show()
#as you can see, NTA area size doesn't show any significant effect on the Food venues count, and R^2 error is somewhere on the floor

In [ ]:
x_train_select=x_train[['Population']]
x_test_select=x_test[['Population']]
lr.fit(x_train_select,y_train)
np.set_printoptions(precision=4,formatter={"float_kind": lambda x: "%g" % x})
print(np.around(lr.coef_,decimals=4))

plt.figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(x_train_select, lr.coef_[0][0]*x_train_select + lr.intercept_[0], '-r')
plt.scatter(df_final_clean['Population'], df_final_clean['Food'], color='blue')
plt.xlabel("Population count")
plt.ylabel("Food Venues count")
print('R^2 for the test data is {}'.format(lr.score(x_test_select, y_test)))
plt.show()
#as you can see, NTA population doesn't show any significant effect on the Food venues count, and R^2 error is also very low

In [ ]:
x_train_select=x_train[['popinlabou']]
x_test_select=x_test[['popinlabou']]
lr.fit(x_train_select,y_train)
np.set_printoptions(precision=4,formatter={"float_kind": lambda x: "%g" % x})
print(np.around(lr.coef_,decimals=4))

plt.figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(x_train_select, lr.coef_[0][0]*x_train_select + lr.intercept_[0], '-r')
plt.scatter(df_final_clean['popinlabou'], df_final_clean['Food'], color='blue')
plt.xlabel("Median income amount")
plt.ylabel("Food Venues count")
print('R^2 for the test data is {}'.format(lr.score(x_test_select, y_test)))
plt.show()
# well, for population that is working, we have a 0.08 R squared, which I will include in our model creation

In [ ]:
x_train_select=x_train[['Shop & Service']]
x_test_select=x_test[['Shop & Service']]
lr.fit(x_train_select,y_train)
np.set_printoptions(precision=4,formatter={"float_kind": lambda x: "%g" % x})
print(np.around(lr.coef_,decimals=4))

plt.figure(num=None, figsize=(16, 6), dpi=80, facecolor='w', edgecolor='k')

plt.plot(x_train_select, lr.coef_[0][0]*x_train_select + lr.intercept_[0], '-r')
plt.scatter(df_final_clean['Shop & Service'], df_final_clean['Food'], color='blue')
plt.xlabel("Shop & Services venue count")
plt.ylabel("Food Venues count")
print('R^2 for the test data is {}'.format(lr.score(x_test_select, y_test)))
plt.show()
# well, for population that is working, we have a almost 0.5 R squared

### Now let's try linear models with x as 'Nightlife Spot' and 'Shop & Service'

In [ ]:
from sklearn.metrics import mean_squared_error

y_data=df_final[['Food']]
x_data=df_final[['Nightlife Spot']]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)

lr.fit(x_train,y_train)
print("The test R squared is equal to", lr.score(x_test,y_test), 'mean squared error is equal to',mean_squared_error(y_test,lr.predict(x_test)))
pd.DataFrame(lr.coef_,columns=x_train.columns)

In [ ]:
y_data=df_final[['Food']]
x_data=df_final[['Shop & Service']]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)

lr.fit(x_train,y_train)
print("The test R squared is equal to", lr.score(x_test,y_test), 'mean squared error is equal to',mean_squared_error(y_test,lr.predict(x_test)))
pd.DataFrame(lr.coef_,columns=x_train.columns)

### Let's create a multilinear Regression

In [ ]:
y_data=df_final[['Food']]
x_data=df_final[['popinlabou','Arts & Entertainment','College & University','Nightlife Spot','Outdoors & Recreation','Professional & Other Places','Residence','Shop & Service','Travel & Transport']]
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=1)

lr.fit(x_train,y_train)
print("The test R squared is equal to", lr.score(x_test,y_test), 'mean squared error is equal to',mean_squared_error(y_test,lr.predict(x_test)))
pd.DataFrame(lr.coef_,columns=x_train.columns)

### Now let's try Ridge regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

pr=PolynomialFeatures(degree=2)
x_train_pr=pr.fit_transform(x_train)
x_test_pr=pr.fit_transform(x_test)

RidgeModel=Ridge(alpha=200000000) # experimenting with various degree and alpha values this looks like the optimal solution
RidgeModel.fit(x_train_pr, y_train)
print("The train R squared is equal to", RidgeModel.score(x_train_pr,y_train), 'mean squared error is equal to',mean_squared_error(y_train,RidgeModel.predict(x_train_pr)))
print("The test R squared is equal to", RidgeModel.score(x_test_pr,y_test), 'mean squared error is equal to',mean_squared_error(y_test,RidgeModel.predict(x_test_pr)))

### And as the last model, let's try to do K-fold for the multilinear Regression

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

# since we've deleted all the unessesary data, we can restore 20 deleted NTA's with NaN values
#x_data=df_final_clean.drop(['NTA Code','Borough','NTA Name','Food'],axis=1)
#x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.15, random_state=1)
#lr.fit(x_train,y_train)
#print('R^2 for the test data is {}'.format(lr.score(x_test, y_test)))
#pd.DataFrame(np.around(lr.coef_,decimals=4),columns=x_train.columns)
Rcross = cross_val_score(lr, x_data, y_data, cv=4)
yhat=cross_val_predict(lr,x_data, y_data,cv=4)
print("The mean of the folds are", Rcross.mean(), "and the mean squared error is" , mean_squared_error(y_data,yhat))

## And the K-fold model shown the best result! let's apply it to the whole dataset to determine which NTA is the most undersaturated

In [ ]:
def minus_func(row):
    return row['Food venues estimate']-row['Food']

df_compare=df_final[['NTA Name','Borough','Food']]
df_compare.loc[:,'Food venues estimate']=cross_val_predict(lr,x_data, y_data,cv=4)
df_compare.loc[:,'Food Venue count diff']=df_compare.apply(minus_func,axis=1)
df_compare.sort_values('Food Venue count diff',ascending=False).head(10)

# So this cross-validated multiple linear regression concludes our task of seing the most undersaturated restaurant Neighborhood tabulation areas in New-york city
### As a bonus, let's build a map using the results that we have

In [ ]:
world_map = folium.Map(location=[40.7128,-74.0060], zoom_start=12)
#please be aware that location=[latitude, longitude] is encoded in folium, however GEOjson files have [longitude,latitude] written down
df_compare.reset_index(inplace=True)
folium.Choropleth(
     name='Food Venues prediction differences',
     geo_data=json_geoinfo,
     data=df_compare,
     columns=['NTA Code', 'Food Venue count diff'],
     key_on='feature.properties.NTACode',
     fill_color='YlOrRd', 
     fill_opacity=0.7, 
     line_opacity=0.2,
     legend_name='Difference between predicted and actual Food Venue Count'
).add_to(world_map)

world_map